In [1]:
from keras.optimizers import SGD
from keras.models import Sequential, Model
from keras.preprocessing.image import ImageDataGenerator
from sklearn.cross_validation import StratifiedKFold
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Flatten, Dense, Dropout, Reshape, Permute, Activation, \
    Input, merge
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from convnetskeras.customlayers import convolution2Dgroup, crosschannelnormalization, \
    splittensor, Softmax4D
from sklearn.metrics import classification_report,confusion_matrix
from keras.utils import np_utils
from sklearn import svm

import sys, glob, argparse
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

import json
import os.path

import h5py as h5
import numpy as np

import hickle as hkl

seed = 7
np.random.seed(seed)
# path to the model weights file.
weights_path = '../dataset/alexnet_weights.h5'
top_model_weights_path = 'bottleneck_fc_model.h5'
nb_train_samples = 10000
nb_validation_samples = 10000
nb_class = 100
nb_epoch = 100
fold_count = 1

def plot_confusion_matrix(cm, title='Confusion matrix', cmap=plt.cm.jet):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(100)
    plt.xticks(tick_marks, ['AisKacang' , 'AngKuKueh' , 'ApamBalik' , 'Asamlaksa' , 'Bahulu' , 'Bakkukteh',
         'BananaLeafRice' , 'Bazhang' , 'BeefRendang' , 'BingkaUbi' , 'Buburchacha',
         'Buburpedas' , 'Capati' , 'Cendol' , 'ChaiTowKuay' , 'CharKuehTiao' , 'CharSiu',
         'CheeCheongFun' , 'ChiliCrab' , 'Chweekueh' , 'ClayPotRice' , 'CucurUdang',
         'CurryLaksa' , 'CurryPuff' , 'Dodol' , 'Durian' , 'DurianCrepe' , 'FishHeadCurry',
         'Guava' , 'HainaneseChickenRice' , 'HokkienMee' , 'Huatkuih' , 'IkanBakar',
         'Kangkung' , 'KayaToast' , 'Keklapis' , 'Ketupat' , 'KuihDadar' , 'KuihLapis',
         'KuihSeriMuka' , 'Langsat' , 'Lekor' , 'Lemang' , 'LepatPisang' , 'LorMee',
         'Maggi goreng' , 'Mangosteen' , 'MeeGoreng' , 'MeeHoonKueh' , 'MeeHoonSoup',
         'MeeJawa' , 'MeeRebus' , 'MeeRojak' , 'MeeSiam' , 'Murtabak' , 'Murukku',
         'NasiGorengKampung' , 'NasiImpit' , 'Nasikandar' , 'Nasilemak' , 'Nasipattaya',
         'Ondehondeh' , 'Otakotak' , 'OysterOmelette' , 'PanMee' , 'PineappleTart',
         'PisangGoreng' , 'Popiah' , 'PrawnMee' , 'Prawnsambal' , 'Puri' , 'PutuMayam',
         'PutuPiring' , 'Rambutan' , 'Rojak' , 'RotiCanai' , 'RotiJala' , 'RotiJohn',
         'RotiNaan' , 'RotiTissue' , 'SambalPetai' , 'SambalUdang' , 'Satay' , 'Sataycelup',
         'SeriMuka' , 'SotoAyam' , 'TandooriChicken' , 'TangYuan' , 'TauFooFah',
         'TauhuSumbat' , 'Thosai' , 'TomYumSoup' , 'Wajik' , 'WanTanMee' , 'WaTanHo' , 'Wonton',
         'YamCake' , 'YongTauFu' , 'Youtiao' , 'Yusheng'], rotation=45)
    plt.yticks(tick_marks, ['AisKacang' , 'AngKuKueh' , 'ApamBalik' , 'Asamlaksa' , 'Bahulu' , 'Bakkukteh',
         'BananaLeafRice' , 'Bazhang' , 'BeefRendang' , 'BingkaUbi' , 'Buburchacha',
         'Buburpedas' , 'Capati' , 'Cendol' , 'ChaiTowKuay' , 'CharKuehTiao' , 'CharSiu',
         'CheeCheongFun' , 'ChiliCrab' , 'Chweekueh' , 'ClayPotRice' , 'CucurUdang',
         'CurryLaksa' , 'CurryPuff' , 'Dodol' , 'Durian' , 'DurianCrepe' , 'FishHeadCurry',
         'Guava' , 'HainaneseChickenRice' , 'HokkienMee' , 'Huatkuih' , 'IkanBakar',
         'Kangkung' , 'KayaToast' , 'Keklapis' , 'Ketupat' , 'KuihDadar' , 'KuihLapis',
         'KuihSeriMuka' , 'Langsat' , 'Lekor' , 'Lemang' , 'LepatPisang' , 'LorMee',
         'Maggi goreng' , 'Mangosteen' , 'MeeGoreng' , 'MeeHoonKueh' , 'MeeHoonSoup',
         'MeeJawa' , 'MeeRebus' , 'MeeRojak' , 'MeeSiam' , 'Murtabak' , 'Murukku',
         'NasiGorengKampung' , 'NasiImpit' , 'Nasikandar' , 'Nasilemak' , 'Nasipattaya',
         'Ondehondeh' , 'Otakotak' , 'OysterOmelette' , 'PanMee' , 'PineappleTart',
         'PisangGoreng' , 'Popiah' , 'PrawnMee' , 'Prawnsambal' , 'Puri' , 'PutuMayam',
         'PutuPiring' , 'Rambutan' , 'Rojak' , 'RotiCanai' , 'RotiJala' , 'RotiJohn',
         'RotiNaan' , 'RotiTissue' , 'SambalPetai' , 'SambalUdang' , 'Satay' , 'Sataycelup',
         'SeriMuka' , 'SotoAyam' , 'TandooriChicken' , 'TangYuan' , 'TauFooFah',
         'TauhuSumbat' , 'Thosai' , 'TomYumSoup' , 'Wajik' , 'WanTanMee' , 'WaTanHo' , 'Wonton',
         'YamCake' , 'YongTauFu' , 'Youtiao' , 'Yusheng'])
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

def visualize_loss(hist):
    train_loss=hist.history['loss']
    val_loss=hist.history['val_loss']
    train_acc=hist.history['acc']
    val_acc=hist.history['val_acc']
    xc=range(nb_epoch)

    plt.figure(1,figsize=(7,5))
    plt.plot(xc,train_loss)
    plt.plot(xc,val_loss)
    plt.xlabel('num of Epochs')
    plt.ylabel('loss')
    plt.title('train_loss vs val_loss')
    plt.grid(True)
    plt.legend(['train','val'])
    #print plt.style.available # use bmh, classic,ggplot for big pictures
    plt.style.use(['classic'])

    plt.figure(2,figsize=(7,5))
    plt.plot(xc,train_acc)
    plt.plot(xc,val_acc)
    plt.xlabel('num of Epochs')
    plt.ylabel('accuracy')
    plt.title('train_acc vs val_acc')
    plt.grid(True)
    plt.legend(['train','val'],loc=4)
    #print plt.style.available # use bmh, classic,ggplot for big pictures
    plt.style.use(['classic'])
    plt.show()

def load_data():
    # load your data using this function
    d = hkl.load('../dataset/myfood100-227.hkl')
    data = d['trainFeatures']
    labels = d['trainLabels']
    lz = d['labels']
    data = data.reshape(data.shape[0], 3, 227, 227)
    #data = data.transpose(0, 2, 3, 1)

    return data,labels,lz

def get_top_model_for_alexnet(nb_class=None, shape=None, W_regularizer=False, weights_file_path=None, input=None, output=None):
    if not output:
        inputs = Input(shape=shape)

    dense_3 = Dense(nb_class,name='dense_3')(inputs)
    predictions = Activation("softmax",name="softmax")(dense_3)
    model = Model(input=input or inputs, output=predictions)

    return model

def load_model(nb_class, weights_path=None):

    inputs = Input(shape=(3,227,227))

    conv_1 = Convolution2D(96, 11, 11,subsample=(4,4),activation='relu',
                           name='conv_1')(inputs)

    conv_2 = MaxPooling2D((3, 3), strides=(2,2))(conv_1)
    conv_2 = crosschannelnormalization(name="convpool_1")(conv_2)
    conv_2 = ZeroPadding2D((2,2))(conv_2)
    conv_2 = merge([
        Convolution2D(128,5,5,activation="relu",name='conv_2_'+str(i+1))(
            splittensor(ratio_split=2,id_split=i)(conv_2)
        ) for i in range(2)], mode='concat',concat_axis=1,name="conv_2")

    conv_3 = MaxPooling2D((3, 3), strides=(2, 2))(conv_2)
    conv_3 = crosschannelnormalization()(conv_3)
    conv_3 = ZeroPadding2D((1,1))(conv_3)
    conv_3 = Convolution2D(384,3,3,activation='relu',name='conv_3')(conv_3)

    conv_4 = ZeroPadding2D((1,1))(conv_3)
    conv_4 = merge([
        Convolution2D(192,3,3,activation="relu",name='conv_4_'+str(i+1))(
            splittensor(ratio_split=2,id_split=i)(conv_4)
        ) for i in range(2)], mode='concat',concat_axis=1,name="conv_4")

    conv_5 = ZeroPadding2D((1,1))(conv_4)
    conv_5 = merge([
        Convolution2D(128,3,3,activation="relu",name='conv_5_'+str(i+1))(
            splittensor(ratio_split=2,id_split=i)(conv_5)
        ) for i in range(2)], mode='concat',concat_axis=1,name="conv_5")

    conv_5 = MaxPooling2D((3, 3), strides=(2,2),name="convpool_5")(conv_5)



    dense_1 = Flatten(name="flatten")(conv_5)
    dense_1 = Dense(4096, activation='relu',name='dense_1')(dense_1)
    dense_2 = Dropout(0.5)(dense_1)
    dense_2 = Dense(4096, activation='relu',name='dense_2')(dense_2)
    dense_3 = Dropout(0.5)(dense_2)
    dense_3 = Dense(nb_class,name='dense_3')(dense_3)
    prediction = Activation("softmax",name="softmax")(dense_3)


    base_model = Model(input=inputs, output=prediction)

    if weights_path:
        base_model.load_weights(weights_path)

    base_model = Model(input=inputs, output=dense_2)

    return base_model



def save_bottlebeck_features(X_train, X_test, y_train, y_test):
    model = load_model(nb_class=nb_class, weights_path=weights_path)
    '''
    j = 0
    for i in X_train:
        temp = X_train[j]
        temp = temp[None, ...]

        bottleneck_features_train.append(model.predict(temp, batch_size=32)[0])
        j+1
    bottleneck_features_train = np.array(bottleneck_features_train)
    np.save(open('alex_bottleneck_features_train.npy', 'wb'), bottleneck_features_train)

    j = 0
    for i in X_test:
        temp = X_train[j]
        temp = temp[None, ...]
        bottleneck_features_validation.append(model.predict(temp, batch_size=32)[0])
        j+1
    bottleneck_features_validation = np.array(bottleneck_features_validation)
    np.save(open('alex_bottleneck_features_validation.npy', 'wb'), bottleneck_features_validation)
    '''
    bottleneck_features_train = model.predict(X_train)
    np.save(open('alex_bottleneck_features_train'+ str(fold_count) +'.npy', 'wb'), bottleneck_features_train)


    bottleneck_features_validation = model.predict(X_test)
    np.save(open('alex_bottleneck_features_validation' + str(fold_count) + '.npy', 'wb'), bottleneck_features_validation)
    print "deep features extracted (x,4096)"


def train_top_model(y_train, y_test):
    X_train = np.load(open('alex_bottleneck_features_train' + str(fold_count) + '.npy' , 'rb'))
    X_test = np.load(open('alex_bottleneck_features_validation' + str(fold_count) + '.npy', 'rb'))

    #svm_train_data = X_train.reshape(nb_train_samples,9216)
    #svm_test_data = X_test.reshape(nb_validation_samples,9216)
    svmy_train = y_train.flatten()
    svmy_test = y_test.flatten()
    
    print "Training SVM.."
    clf = svm.SVC(kernel='linear')

    clf.fit(X_train, svmy_train)
    score = clf.score(X_test, svmy_test)
    print("SVM %s: %.2f%%" % ("acc: ", score*100))
    
    # Compute confusion matrix
    y_pred = clf.predict(X_test)
    target_names = ['AisKacang' , 'AngKuKueh' , 'ApamBalik' , 'Asamlaksa' , 'Bahulu' , 'Bakkukteh',
     'BananaLeafRice' , 'Bazhang' , 'BeefRendang' , 'BingkaUbi' , 'Buburchacha',
     'Buburpedas' , 'Capati' , 'Cendol' , 'ChaiTowKuay' , 'CharKuehTiao' , 'CharSiu',
     'CheeCheongFun' , 'ChiliCrab' , 'Chweekueh' , 'ClayPotRice' , 'CucurUdang',
     'CurryLaksa' , 'CurryPuff' , 'Dodol' , 'Durian' , 'DurianCrepe' , 'FishHeadCurry',
     'Guava' , 'HainaneseChickenRice' , 'HokkienMee' , 'Huatkuih' , 'IkanBakar',
     'Kangkung' , 'KayaToast' , 'Keklapis' , 'Ketupat' , 'KuihDadar' , 'KuihLapis',
     'KuihSeriMuka' , 'Langsat' , 'Lekor' , 'Lemang' , 'LepatPisang' , 'LorMee',
     'Maggi goreng' , 'Mangosteen' , 'MeeGoreng' , 'MeeHoonKueh' , 'MeeHoonSoup',
     'MeeJawa' , 'MeeRebus' , 'MeeRojak' , 'MeeSiam' , 'Murtabak' , 'Murukku',
     'NasiGorengKampung' , 'NasiImpit' , 'Nasikandar' , 'Nasilemak' , 'Nasipattaya',
     'Ondehondeh' , 'Otakotak' , 'OysterOmelette' , 'PanMee' , 'PineappleTart',
     'PisangGoreng' , 'Popiah' , 'PrawnMee' , 'Prawnsambal' , 'Puri' , 'PutuMayam',
     'PutuPiring' , 'Rambutan' , 'Rojak' , 'RotiCanai' , 'RotiJala' , 'RotiJohn',
     'RotiNaan' , 'RotiTissue' , 'SambalPetai' , 'SambalUdang' , 'Satay' , 'Sataycelup',
     'SeriMuka' , 'SotoAyam' , 'TandooriChicken' , 'TangYuan' , 'TauFooFah',
     'TauhuSumbat' , 'Thosai' , 'TomYumSoup' , 'Wajik' , 'WanTanMee' , 'WaTanHo' , 'Wonton',
     'YamCake' , 'YongTauFu' , 'Youtiao' , 'Yusheng']
    cm = confusion_matrix(svmy_test, y_pred)
    print cm
    print(classification_report(svmy_test, y_pred,target_names=target_names))
    np.set_printoptions(precision=2)
    plt.figure()
    plot_confusion_matrix(cm)
    plt.show()


    print "\nTraining CNN.."
    y_train = np_utils.to_categorical(y_train, nb_class)
    y_test = np_utils.to_categorical(y_test, nb_class)

    shape=X_train.shape[1:]
    
    model = None # Clear Model

    model = get_top_model_for_alexnet(
        shape=shape,
        nb_class=100)

    model.compile(
        loss='categorical_crossentropy',
        optimizer=SGD(lr=0.0001, decay=1e-6, momentum=0.9, nesterov=True),
        metrics=['accuracy'])

    hist = model.fit(X_train, y_train,
              nb_epoch=nb_epoch, batch_size=32,verbose=0)
    
    #visualize_loss(hist) # Removing this because we not training CNN, just classification 

    scores = model.evaluate(X_test, y_test, verbose=0)
    print("CNN %s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    
    '''
    y_proba = model.predict(X_test)
    y_pred = np_utils.probas_to_classes(y_proba)

    target_names = ['AisKacang' , 'AngKuKueh' , 'ApamBalik' , 'Asamlaksa' , 'Bahulu' , 'Bakkukteh',
                     'BananaLeafRice' , 'Bazhang' , 'BeefRendang' , 'BingkaUbi']
    cm = confusion_matrix(np.argmax(y_test,axis=1), y_pred)
    print(classification_report(np.argmax(y_test,axis=1), y_pred,target_names=target_names))
    print cm
    
    # Visualization of confusion matrix
    np.set_printoptions(precision=2)
    plt.figure()
    plot_confusion_matrix(cm)
    plt.show()
    '''

    model.save_weights('model/deepfeatures_weights' + str(fold_count) + '.h5')
    return score

if __name__ == "__main__":
    n_folds = 2
    total_scores = 0
    
    print "Loading data.."
    data, labels, lz = load_data()
    data = data.astype('float32')
    data /= 255
    lz = np.array(lz)
    print lz.shape
    print "Data loaded !"
    
    skf = StratifiedKFold(y=lz, n_folds=n_folds, shuffle=False)
    
    for i, (train, test) in enumerate(skf):
        print "Test train Shape: "
        print data[train].shape
        print data[test].shape
        print ("Running Fold %d / %d" % (i+1, n_folds))
        
        save_bottlebeck_features(data[train], data[test],labels[train], labels[test])
        scores = train_top_model(labels[train], labels[test])
        total_scores = total_scores + scores
        fold_count = fold_count + 1
    print("Average acc : %.2f%%" % (total_scores/n_folds*100))

    


Using Theano backend.
Using gpu device 0: GeForce GTX 1060 6GB (CNMeM is disabled, cuDNN 5105)

Loading data..
(20000,)
Data loaded !
Test train Shape: 
(10000, 3, 227, 227)
(10000, 3, 227, 227)


/home/machine/anaconda2/lib/python2.7/site-packages/keras/backend/theano_backend.py:1500: UserWarning: DEPRECATION: the 'ds' parameter is not going to exist anymore as it is going to be replaced by the parameter 'ws'.
  mode='max')



Running Fold 1 / 2
deep features extracted (x,4096)
Training SVM..
SVM acc: : 33.43%
[[49  0  2 ...,  0  0  0]
 [ 0 21  0 ...,  0  1  0]
 [ 1  2 41 ...,  0  0  0]
 ..., 
 [ 0  0  0 ..., 48  0  0]
 [ 0  3  1 ...,  0 15  0]
 [ 3  0  0 ...,  0  0 28]]
                      precision    recall  f1-score   support

           AisKacang       0.45      0.49      0.47       100
           AngKuKueh       0.15      0.21      0.18       100
           ApamBalik       0.34      0.41      0.37       100
           Asamlaksa       0.19      0.21      0.20       100
              Bahulu       0.58      0.69      0.63       100
           Bakkukteh       0.43      0.48      0.45       100
      BananaLeafRice       0.38      0.34      0.36       100
             Bazhang       0.19      0.23      0.21       100
         BeefRendang       0.54      0.76      0.63       100
           BingkaUbi       0.42      0.45      0.43       100
         Buburchacha       0.33      0.36      0.34       100
     

/home/machine/anaconda2/lib/python2.7/site-packages/keras/backend/theano_backend.py:1500: UserWarning: DEPRECATION: the 'st' parameter is not going to exist anymore as it is going to be replaced by the parameter 'stride'.
  mode='max')
/home/machine/anaconda2/lib/python2.7/site-packages/keras/backend/theano_backend.py:1500: UserWarning: DEPRECATION: the 'padding' parameter is not going to exist anymore as it is going to be replaced by the parameter 'pad'.
  mode='max')
